In [23]:
from regression import add_neighborhood_feature, avg_loss, SGD, sigmoid
import numpy as np
import matplotlib.pyplot as plt
import random
from matplotlib.colors import ListedColormap, BoundaryNorm

In [24]:
training_data = np.load('1k_data/data.npy')
training_labels = np.load('1k_data/labels.npy')
datatesting_data = np.load('Task1_Testset500/data.npy')
testing_labels = np.load('Task1_Testset500/labels.npy')
data = np.load('1k_data/data.npy')
labels = np.load('1k_data/labels.npy')
# wrong data rn

In [25]:
def SGD2(w,x,y,alpha):
    y_prime = softmax(x)
    cur_loss = categorical_cross_entropy(y, y_prime) + (1e-10 * np.sum(np.abs(w))) # l2 reg can be changed for the model)
    w -= alpha * (y_prime - y)

def softmax(x):
    exp_x = np.exp(x - np.max(x))  # Subtracting np.max(x) for numerical stability
    return exp_x / exp_x.sum(axis=0, keepdims=True)

def categorical_cross_entropy(y_actual, y_prime):
    epsilon = 1e-15  # Small constant to avoid taking the logarithm of zero
    y_prime = np.clip(y_prime, epsilon, 1 - epsilon)  # Clip to avoid numerical instability
    return -np.mean(np.sum(y_actual * np.log(y_prime), axis=1))

def feed_forward_multiclass(data, labels, num_iters=1000):
    data = np.column_stack([data, np.ones(data.shape[0])]) # Adding in a bias term
    samples,features = data.shape
    w = np.random.rand(features)*2 - np.ones(shape=features) #[0,1] -> [-100,100]
    loss_coords, accuracy_coords = [], []
    for i in range(num_iters):
        index = np.random.randint(0, 1000)
        x = data[index]
        y = labels[index][0]
        loss = SGD(w, x, y, alpha=0.01)
        loss_coords.append(avg_loss(data,w,labels))
        accuracy_coords.append(accuracy(data,labels,w))
          
    print(f'Average Loss: {avg_loss(data,w,labels)}, Accuracy on testing set: {accuracy(data, labels, w)}')
    return (loss_coords, accuracy_coords)

data = add_neighborhood_feature(training_data)
data = add_polynomial_features(data, degree=4)
loss_coords, accuracy_coords = feed_forward_multiclass(data, labels)


Average Loss: -1.1102230246251563e-15, Accuracy on testing set: 1.0


In [26]:
def add_polynomial_features(X, degree):
    """Add polynomial features to the input data."""
    features = [X]
    for d in range(2, degree + 1):
        features.append(X**d)
    return np.concatenate(features, axis=1)

def feed_forward(data,labels, num_iters = 10000):
    # Adding in a bias term
    bias = np.ones((data.shape[0], 1))
    data = np.concatenate((data, bias), axis=1)
    # print(data.shape)
    samples,features = data.shape
    loss_coords, accuracy_coords = [], []
    w = np.random.rand(features)*2 - np.ones(shape=features) #[0,1] -> [-1,1]
    for i in range(num_iters):
        index = np.random.randint(0, 1000)
        x = data[index]
        y = labels[index][0]
        loss = SGD(w, x, y, alpha=0.01)
        if i % 100 == 0:
            loss_coords.append(avg_loss(data,w,labels))
            accuracy_coords.append(accuracy(data,labels,w))
        
    print(f'Average Loss: {avg_loss(data,w,labels)}, Accuracy on testing set: {accuracy(data, labels, w)}')
    return (loss_coords, accuracy_coords)

def accuracy(data, labels, weights):
    # print(data.shape)
    outs = sigmoid(np.dot(data, weights))
    binary = np.array((outs >= 0.5).astype(int))
    samples = data.shape[0]
    correct = np.sum(np.equal(binary,labels[:,0]))
    return correct/samples